In [1]:
# data clean packages
import pandas as pd
import numpy as np
import datetime as dt

# plot packages
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

# network packages
import networkx as nx
from networkx.algorithms import approximation
from networkx.algorithms import community
import community
import collections

In [2]:
df_customers = pd.read_pickle('customers_clean.pkl')
df_suppliers = pd.read_pickle('suppliers_clean.pkl')
df = pd.concat([df_customers, df_suppliers])

In [3]:
df_suppliers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63236 entries, 0 to 63235
Data columns (total 8 columns):
id               63236 non-null object
start            63236 non-null datetime64[ns]
end              63236 non-null datetime64[ns]
rel_type         63236 non-null object
source_ticker    63236 non-null object
target_ticker    63236 non-null object
source_cusip     63236 non-null object
target_cusip     63236 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 3.9+ MB


In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164263 entries, 0 to 164262
Data columns (total 8 columns):
id               164263 non-null object
start            164263 non-null datetime64[ns]
end              164263 non-null datetime64[ns]
rel_type         164263 non-null object
source_ticker    164263 non-null object
target_ticker    164263 non-null object
source_cusip     164263 non-null object
target_cusip     164263 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 10.0+ MB


In [5]:
# df['start']=pd.to_datetime(df['start']) #convert datadate to date fmt
# df['end']=pd.to_datetime(df['end'])

In [6]:
df = df[df['start'] != df['end']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226909 entries, 0 to 63235
Data columns (total 8 columns):
id               226909 non-null object
start            226909 non-null datetime64[ns]
end              226909 non-null datetime64[ns]
rel_type         226909 non-null object
source_ticker    226909 non-null object
target_ticker    226909 non-null object
source_cusip     226909 non-null object
target_cusip     226909 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 15.6+ MB


In [8]:
df.head()

,id,start,end,rel_type,source_ticker,target_ticker,source_cusip,target_cusip
0,CUST-100000247,2016-11-03,2017-08-10,CUSTOMER,IW5-SG,ADRO-ID,Y7693G104,Y7087B109
1,CUST-100000500,2016-11-03,2017-08-10,CUSTOMER,IW5-SG,INTP-ID,Y7693G104,Y7127B135
2,CUST-100001093,2016-11-03,2017-08-10,CUSTOMER,OANDO-NG,XOM,V6796A108,30231G102
3,CUST-100001232,2016-11-03,2017-08-10,CUSTOMER,533278-IN,500470-IN,Y1668L107,Y8547N139
4,CUST-100001451,2016-11-03,2017-08-10,CUSTOMER,533278-IN,526371-IN,Y1668L107,Y6223W100


In [9]:
df2016 = df.loc[(df['end'] > '2015-12-31') & (df['start'] < '2017-01-01')]
df2015 = df.loc[(df['end'] > '2014-12-31') & (df['start'] < '2016-01-01')]

In [10]:
df2015.to_pickle('df2015.pkl')
df2016.to_pickle('df2016.pkl')

In [11]:
df2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103475 entries, 0 to 63235
Data columns (total 8 columns):
id               103475 non-null object
start            103475 non-null datetime64[ns]
end              103475 non-null datetime64[ns]
rel_type         103475 non-null object
source_ticker    103475 non-null object
target_ticker    103475 non-null object
source_cusip     103475 non-null object
target_cusip     103475 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 7.1+ MB


In [12]:
df2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87210 entries, 4771 to 59742
Data columns (total 8 columns):
id               87210 non-null object
start            87210 non-null datetime64[ns]
end              87210 non-null datetime64[ns]
rel_type         87210 non-null object
source_ticker    87210 non-null object
target_ticker    87210 non-null object
source_cusip     87210 non-null object
target_cusip     87210 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 6.0+ MB


# Undirected graph centrality

In [29]:
# list = [df2015]
# net_centrality = pd.DataFrame()
# for item in list:
#     G = nx.from_pandas_edgelist(item, 'source_ticker', 'target_ticker', create_using = nx.DiGraph())
#     # degree centrality
#     degree_centrality = pd.Series(nx.out_degree_centrality(G))
#     degree_centrality = pd.DataFrame(degree_centrality, columns = ['degree_centrality'])
    
#     # closeness
#     closeness_centrality = pd.Series(nx.closeness_centrality(G))
#     closeness_centrality = pd.DataFrame(closeness_centrality, columns = ['closeness_centrality'])
    
#     # betweenness
#     betweenness_centrality = pd.Series(nx.betweenness_centrality(G, normalized=True))
#     betweenness_centrality = pd.DataFrame(betweenness_centrality, columns=['betweenness_centrality'])
    
#     #eigenvector
#     eigenvector_centrality = pd.Series(nx.eigenvector_centrality(G, max_iter=100, tol=1e-06, nstart=None, weight=None))
#     eigenvector_centrality = pd.DataFrame(eigenvector_centrality, columns=['eigenvector_centrality'])
    
#     #katz_centrality
#     katz_centrality = pd.Series(nx.katz_centrality(G, alpha=0.1, beta=1.0, max_iter=1000, tol=1e-06, nstart=None, normalized=True, weight=None))
#     katz_centrality = pd.DataFrame(katz_centrality, columns=['katz_centrality'])
    
#     # harmonic_centrality
#     harmonic_centrality = pd.Series(nx.harmonic_centrality(G))
#     harmonic_centrality = pd.DataFrame(harmonic_centrality, columns = ['harmonic_centrality'])
    
#     #merge_time
#     data_merge_time = pd.concat([degree_centrality, closeness_centrality, betweenness_centrality, eigenvector_centrality],axis =1)
#     data_merge_time['date'] = " ".join(str(x) for x in item['date'].unique())
    
#     #merge_all
#     net_centrality = data_merge_time.append(net_centrality)

# Undirected graph net features

In [13]:
G = nx.from_pandas_edgelist(df2015, 'source_ticker', 'target_ticker', create_using = nx.Graph())

node_clique_number = pd.Series(nx.node_clique_number(G))
node_clique_number = pd.DataFrame(node_clique_number, columns = ['node_clique_number'])

load_centrality = pd.Series(nx.load_centrality(G))
load_centrality = pd.DataFrame(load_centrality, columns = ['load_centrality'])

number_of_cliques = pd.Series(nx.number_of_cliques(G))
number_of_cliques = pd.DataFrame(number_of_cliques, columns = ['load_centrality'])

triangles = pd.Series(nx.triangles(G))
triangles = pd.DataFrame(triangles, columns = ['triangles'])

clustering = pd.Series(nx.clustering(G))
clustering = pd.DataFrame(clustering, columns = ['triangles'])

undirected = pd.concat([node_clique_number,load_centrality,number_of_cliques,triangles,clustering],axis =1)

In [16]:
undirected.columns = ['node_clique_number','load_centrality','number_of_cliques','triangles','clustering']

In [17]:
undirected.head()

,node_clique_number,load_centrality,number_of_cliques,triangles,clustering
LOGN-CH,4,0.000398,55,39,0.039394
IM,6,0.000960,132,171,0.075066
TECD,6,0.000804,137,179,0.086058
BBY,5,0.001817,149,164,0.025019
WMT,6,0.027733,700,734,0.008263


In [19]:
undirected.describe().T

,count,mean,std,min,25%,50%,75%,max
node_clique_number,16634.0,2.597331,0.933325,2.0,2.0,2.000000,3.000000,8.00000
load_centrality,16634.0,0.000190,0.000937,0.0,0.0,0.000007,0.000122,0.05508
number_of_cliques,16634.0,11.133702,42.236055,1.0,1.0,3.000000,8.000000,1519.00000
triangles,16634.0,8.897018,51.682485,0.0,0.0,0.000000,2.000000,1861.00000
clustering,16634.0,0.057022,0.143069,0.0,0.0,0.000000,0.054545,1.00000


In [18]:
undirected.to_pickle('undirected.pkl')

# Directed graph net centrality

In [16]:
list = [df2015]
net_centrality = pd.DataFrame()
for item in list:
    G = nx.from_pandas_edgelist(item, 'source_ticker', 'target_ticker', create_using = nx.DiGraph())
    G.remove_edges_from(nx.selfloop_edges(G))
    # out_degree centrality
    out_degree_centrality = pd.Series(nx.out_degree_centrality(G))
    out_degree_centrality = pd.DataFrame(out_degree_centrality, columns = ['out_degree_centrality'])
    
    # in_degree centrality
    in_degree_centrality = pd.Series(nx.in_degree_centrality(G))
    in_degree_centrality = pd.DataFrame(in_degree_centrality, columns = ['in_degree_centrality'])
    
    
    # closeness
    closeness_centrality = pd.Series(nx.closeness_centrality(G))
    closeness_centrality = pd.DataFrame(closeness_centrality, columns = ['closeness_centrality'])
    
    # betweenness
    betweenness_centrality = pd.Series(nx.betweenness_centrality(G, normalized=True))
    betweenness_centrality = pd.DataFrame(betweenness_centrality, columns=['betweenness_centrality'])
    
    # eigenvector
    eigenvector_centrality = pd.Series(nx.eigenvector_centrality(G, max_iter=100, tol=1e-06, nstart=None, weight=None))
    eigenvector_centrality = pd.DataFrame(eigenvector_centrality, columns=['eigenvector_centrality'])
    
    # harmonic_centrality
    harmonic_centrality = pd.Series(nx.harmonic_centrality(G))
    harmonic_centrality = pd.DataFrame(harmonic_centrality, columns = ['harmonic_centrality'])
    
    # merge_time
    data_merge_time = pd.concat([out_degree_centrality, in_degree_centrality, closeness_centrality, 
                                 betweenness_centrality, eigenvector_centrality, harmonic_centrality],axis =1)
    #data_merge_time['date'] = " ".join(str(x) for x in item['date'].unique())
    
    # merge_all
    net_centrality = data_merge_time.append(net_centrality)
    net_centrality.to_pickle('net_centrality.pkl')

In [17]:
net_centrality.head()

,out_degree_centrality,in_degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,harmonic_centrality
LOGN-CH,0.002044,0.000721,0.122133,0.000145,0.006193,2183.177778
IM,0.000782,0.003547,0.155451,0.000665,0.073388,2883.121429
TECD,0.002645,0.001984,0.142762,0.002240,0.036922,2614.630952
BBY,0.000421,0.006673,0.156793,0.000711,0.077027,2933.089683
WMT,0.001744,0.024109,0.183382,0.012311,0.155350,3527.580952


In [18]:
net_centrality.describe()

,out_degree_centrality,in_degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,harmonic_centrality
count,16634.000000,16634.000000,16634.000000,16634.000000,1.663400e+04,16634.000000
mean,0.000293,0.000293,0.067250,0.000095,2.417988e-03,1186.310715
std,0.000621,0.000943,0.055905,0.000563,7.367118e-03,993.652334
min,0.000000,0.000000,0.000000,0.000000,1.246922e-24,0.000000
25%,0.000000,0.000060,0.000060,0.000000,2.369152e-23,1.000000
50%,0.000060,0.000060,0.093677,0.000000,5.933571e-05,1618.776515
75%,0.000301,0.000240,0.114815,0.000035,1.401326e-03,2024.018687
max,0.018457,0.024710,0.183382,0.029359,1.553502e-01,3527.580952


In [48]:
print(net_centrality.corr().round(2).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  out\_degree\_centrality &  in\_degree\_centrality &  closeness\_centrality &  betweenness\_centrality &  eigenvector\_centrality &  harmonic\_centrality \\
\midrule
out\_degree\_centrality  &                   1.00 &                  0.48 &                  0.13 &                    0.64 &                    0.33 &                 0.14 \\
in\_degree\_centrality   &                   0.48 &                  1.00 &                  0.33 &                    0.71 &                    0.70 &                 0.35 \\
closeness\_centrality   &                   0.13 &                  0.33 &                  1.00 &                    0.20 &                    0.41 &                 1.00 \\
betweenness\_centrality &                   0.64 &                  0.71 &                  0.20 &                    1.00 &                    0.52 &                 0.22 \\
eigenvector\_centrality &                   0.33 &                  0.70 &                  0

# Directed graph network features

In [24]:
list = [df2015]
net_features_direct = pd.DataFrame()
for item in list:
    G = nx.from_pandas_edgelist(item, 'source_ticker', 'target_ticker', create_using = nx.DiGraph())
    G.remove_edges_from(nx.selfloop_edges(G))
    # average_neighbor_degree
    average_neighbor_degree = pd.Series(nx.average_neighbor_degree(G))
    average_neighbor_degree = pd.DataFrame(average_neighbor_degree, columns = ['average_neighbor_degree'])
    
    # clustering
#     clustering = pd.Series(nx.clustering(G))
#     clustering = pd.DataFrame(clustering, columns = ['clustering'])
    
    #square_clustering 
    square_clustering = pd.Series(nx.square_clustering(G))
    square_clustering = pd.DataFrame(square_clustering, columns = ['square_clustering'])
    
    # core_number 
    core_number = pd.Series(nx.core_number(G))
    core_number = pd.DataFrame(core_number, columns = ['core_number'])
    
    # pagerank
    pagerank = pd.Series(nx.pagerank(G, alpha = 0.9))
    pagerank = pd.DataFrame(pagerank, columns = ['pagerank'])
    
    # hubs and authorties
    hits = nx.hits(G, max_iter=500)
    
    hubs = pd.Series(hits[0])
    hubs = pd.DataFrame(hubs, columns = ['hubs'])
    
    authorities = pd.Series(hits[1])
    authorities = pd.DataFrame(authorities, columns = ['authorities'])
    
    
    #merge_time
    data_merge_time = pd.concat([average_neighbor_degree,square_clustering, core_number, pagerank,hubs, authorities],axis =1)
    #data_merge_time['date'] = " ".join(str(x) for x in item['date'].unique())
    
    #merge_all
    net_features_direct = data_merge_time.append(net_features_direct)
    net_features_direct.to_pickle('net_features_direct.pkl')

In [25]:
net_features_direct.head()

,average_neighbor_degree,square_clustering,core_number,pagerank,hubs,authorities
LOGN-CH,8.588235,0.002522,19,0.000043,0.000342,0.000204
IM,6.461538,0.002287,19,0.000426,0.000044,0.001083
TECD,52.545455,0.002850,19,0.000147,0.000569,0.000588
BBY,23.285714,0.002941,19,0.000543,0.000046,0.001987
WMT,8.103448,0.002479,19,0.003175,0.000064,0.004445


In [27]:
net_features_direct.describe().T

,count,mean,std,min,25%,50%,75%,max
average_neighbor_degree,16634.0,11.446079,20.289581,0.000000,0.000000e+00,3.000000e+00,15.239518,230.000000
square_clustering,16634.0,0.035802,0.499201,0.000000,0.000000e+00,0.000000e+00,0.001141,23.000000
core_number,16634.0,5.057653,4.976538,1.000000,1.000000e+00,3.000000e+00,7.000000,19.000000
pagerank,16634.0,0.000060,0.000112,0.000019,1.947355e-05,2.609531e-05,0.000053,0.003175
hubs,16634.0,0.000060,0.000181,0.000000,0.000000e+00,9.670547e-07,0.000029,0.002818
authorities,16634.0,0.000060,0.000333,0.000000,1.245861e-167,3.324200e-06,0.000027,0.011560


In [49]:
print(net_features_direct.corr().round(2).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  average\_neighbor\_degree &  square\_clustering &  core\_number &  pagerank &  hubs &  authorities \\
\midrule
average\_neighbor\_degree &                     1.00 &              -0.01 &         0.29 &     -0.02 &  0.34 &         0.01 \\
square\_clustering       &                    -0.01 &               1.00 &         0.02 &      0.00 & -0.01 &        -0.00 \\
core\_number             &                     0.29 &               0.02 &         1.00 &      0.40 &  0.58 &         0.35 \\
pagerank                &                    -0.02 &               0.00 &         0.40 &      1.00 &  0.09 &         0.58 \\
hubs                    &                     0.34 &              -0.01 &         0.58 &      0.09 &  1.00 &         0.16 \\
authorities             &                     0.01 &              -0.00 &         0.35 &      0.58 &  0.16 &         1.00 \\
\bottomrule
\end{tabular}



In [36]:
net_features_direct.to_pickle('net_features_direct.pkl')

In [17]:
net_features = pd.read_pickle('net_features_direct.pkl')
net_features = net_features.drop(columns = ['constraint'])

In [18]:
net_centrality = pd.read_pickle('net_centrality.pkl')
net_centrality = net_centrality.dropna()

In [63]:
net_centrality.head()

,out_degree_centrality,in_degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,katz_centrality,harmonic_centrality
TRC,0.000470,0.000000,0.000000,0.000000,3.261317e-53,0.004597,0.000000
CPN,0.001035,0.000470,0.000484,0.000069,1.217123e-49,0.006942,5.500000
NCC-AE,0.000000,0.000094,0.000094,0.000000,2.706893e-51,0.005057,1.000000
DG,0.000000,0.001599,0.020765,0.000000,2.040369e-04,0.013380,256.813013
GVA,0.000000,0.000094,0.000094,0.000000,2.706893e-51,0.005057,1.000000


In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [29]:
minmax = MinMaxScaler()
Standard = preprocessing.StandardScaler()
normalize = preprocessing.Normalizer()

In [31]:
# minmax
net_centrality_minmax = minmax.fit_transform(net_centrality)
net_features_minmax = minmax.fit_transform(net_features_direct)
net_centrality_minmax = pd.DataFrame(net_centrality_minmax)
net_features_minmax = pd.DataFrame(net_features_minmax)
net_centrality_minmax.columns = ['out_degree_centrality','in_degree_centrality','closeness_centrality',
                               'betweenness_centrality','eigenvector_centrality','harmonic_centrality']
net_features_minmax.columns = ['average_neighbor_degree','square_clustering','core_number','pagerank','hubs','authorities']
net_centrality_minmax.describe().round(4).T
#net_features_minmax.head()

,count,mean,std,min,25%,50%,75%,max
out_degree_centrality,16634.0,0.0159,0.0336,0.0,0.0000,0.0033,0.0163,1.0
in_degree_centrality,16634.0,0.0119,0.0381,0.0,0.0024,0.0024,0.0097,1.0
closeness_centrality,16634.0,0.3667,0.3049,0.0,0.0003,0.5108,0.6261,1.0
betweenness_centrality,16634.0,0.0032,0.0192,0.0,0.0000,0.0000,0.0012,1.0
eigenvector_centrality,16634.0,0.0156,0.0474,0.0,0.0000,0.0004,0.0090,1.0
harmonic_centrality,16634.0,0.3363,0.2817,0.0,0.0003,0.4589,0.5738,1.0


In [75]:
print(net_centrality_minmax.describe().round(4).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &    count &    mean &     std &  min &  25\% &     50\% &     75\% &  max \\
\midrule
out\_degree\_centrality  &  10633.0 &  0.0313 &  0.0694 &  0.0 &  0.0 &  0.0000 &  0.0286 &  1.0 \\
in\_degree\_centrality   &  10633.0 &  0.0123 &  0.0397 &  0.0 &  0.0 &  0.0037 &  0.0075 &  1.0 \\
closeness\_centrality   &  10633.0 &  0.0758 &  0.1606 &  0.0 &  0.0 &  0.0016 &  0.0090 &  1.0 \\
betweenness\_centrality &  10633.0 &  0.0021 &  0.0218 &  0.0 &  0.0 &  0.0000 &  0.0000 &  1.0 \\
eigenvector\_centrality &  10633.0 &  0.0025 &  0.0230 &  0.0 &  0.0 &  0.0000 &  0.0000 &  1.0 \\
katz\_centrality        &  10633.0 &  0.0113 &  0.0383 &  0.0 &  0.0 &  0.0025 &  0.0068 &  1.0 \\
harmonic\_centrality    &  10633.0 &  0.0599 &  0.1289 &  0.0 &  0.0 &  0.0011 &  0.0076 &  1.0 \\
\bottomrule
\end{tabular}



In [76]:
print(net_features_minmax.describe().round(4).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &    count &    mean &     std &  min &     25\% &     50\% &     75\% &  max \\
\midrule
average\_neighbor\_degree &  16482.0 &  0.0127 &  0.0340 &  0.0 &  0.0000 &  0.0000 &  0.0116 &  1.0 \\
square\_clustering       &  16482.0 &  0.0008 &  0.0115 &  0.0 &  0.0000 &  0.0000 &  0.0000 &  1.0 \\
core\_number             &  16482.0 &  0.1960 &  0.2735 &  0.0 &  0.0000 &  0.0476 &  0.2857 &  1.0 \\
pagerank                &  16482.0 &  0.0082 &  0.0315 &  0.0 &  0.0001 &  0.0010 &  0.0049 &  1.0 \\
hubs                    &  16482.0 &  0.0155 &  0.0453 &  0.0 &  0.0000 &  0.0000 &  0.0068 &  1.0 \\
authorities             &  16482.0 &  0.0076 &  0.0358 &  0.0 &  0.0000 &  0.0006 &  0.0034 &  1.0 \\
\bottomrule
\end{tabular}



In [45]:
# standard
net_centrality_standard = Standard.fit_transform(net_centrality)
net_features_standard = Standard.fit_transform(net_features_direct)
net_centrality_standard = pd.DataFrame(net_centrality_standard)
net_features_standard = pd.DataFrame(net_features_standard)
net_centrality_standard.columns = ['out_degree_centrality','in_degree_centrality','closeness_centrality',
                               'betweenness_centrality','eigenvector_centrality','harmonic_centrality']
net_features_standard.columns = ['average_neighbor_degree','square_clustering','core_number','pagerank','hubs','authorities']

In [46]:
print(net_centrality_standard.describe().round(2).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &    count &  mean &  std &   min &   25\% &   50\% &   75\% &    max \\
\midrule
out\_degree\_centrality  &  16634.0 &   0.0 &  1.0 & -0.47 & -0.47 & -0.38 &  0.01 &  29.25 \\
in\_degree\_centrality   &  16634.0 &   0.0 &  1.0 & -0.31 & -0.25 & -0.25 & -0.06 &  25.90 \\
closeness\_centrality   &  16634.0 &   0.0 &  1.0 & -1.20 & -1.20 &  0.47 &  0.85 &   2.08 \\
betweenness\_centrality &  16634.0 &  -0.0 &  1.0 & -0.17 & -0.17 & -0.17 & -0.11 &  51.96 \\
eigenvector\_centrality &  16634.0 &   0.0 &  1.0 & -0.33 & -0.33 & -0.32 & -0.14 &  20.76 \\
harmonic\_centrality    &  16634.0 &   0.0 &  1.0 & -1.19 & -1.19 &  0.44 &  0.84 &   2.36 \\
\bottomrule
\end{tabular}



In [47]:
print(net_features_standard.describe().round(2).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &    count &  mean &  std &   min &   25\% &   50\% &   75\% &    max \\
\midrule
average\_neighbor\_degree &  16634.0 &   0.0 &  1.0 & -0.56 & -0.56 & -0.42 &  0.19 &  10.77 \\
square\_clustering       &  16634.0 &   0.0 &  1.0 & -0.07 & -0.07 & -0.07 & -0.07 &  46.00 \\
core\_number             &  16634.0 &  -0.0 &  1.0 & -0.82 & -0.82 & -0.41 &  0.39 &   2.80 \\
pagerank                &  16634.0 &  -0.0 &  1.0 & -0.37 & -0.36 & -0.30 & -0.06 &  27.87 \\
hubs                    &  16634.0 &  -0.0 &  1.0 & -0.33 & -0.33 & -0.33 & -0.17 &  15.27 \\
authorities             &  16634.0 &  -0.0 &  1.0 & -0.18 & -0.18 & -0.17 & -0.10 &  34.59 \\
\bottomrule
\end{tabular}



In [35]:
# normalize
net_centrality_normalize = normalize.fit_transform(net_centrality)
net_features_normalize = normalize.fit_transform(net_features_direct)
net_centrality_normalize = pd.DataFrame(net_centrality_normalize)
net_features_normalize = pd.DataFrame(net_features_normalize)

In [38]:
net_centrality_normalize.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
0,16634.0,0.25,0.43,0.0,0.0,0.0,0.0,1.0
1,16634.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
2,16634.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
3,16634.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
4,16634.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
5,16634.0,0.75,0.43,0.0,1.0,1.0,1.0,1.0


In [40]:
# log transform
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p)

In [41]:
net_centrality_log = transformer.fit_transform(net_centrality)
net_features_log = transformer.fit_transform(net_features_direct)
net_centrality_log = pd.DataFrame(net_centrality_log)
net_features_log = pd.DataFrame(net_features_log)

In [42]:
net_centrality_log.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
0,16634.0,0.00,0.00,0.0,0.00,0.00,0.00,0.02
1,16634.0,0.00,0.00,0.0,0.00,0.00,0.00,0.02
2,16634.0,0.06,0.05,0.0,0.00,0.09,0.11,0.17
3,16634.0,0.00,0.00,0.0,0.00,0.00,0.00,0.03
4,16634.0,0.00,0.01,0.0,0.00,0.00,0.00,0.14
5,16634.0,4.71,3.56,0.0,0.69,7.39,7.61,8.17


In [23]:
net_features_minmax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16482 entries, 0 to 16481
Data columns (total 6 columns):
0    16482 non-null float64
1    16482 non-null float64
2    16482 non-null float64
3    16482 non-null float64
4    16482 non-null float64
5    16482 non-null float64
dtypes: float64(6)
memory usage: 772.7 KB


In [61]:
net_features_minmax.describe().round(2)

,0,1,2,3,4,5
count,16482.00,16482.00,16482.00,16482.00,16482.00,16482.00
mean,0.01,0.00,0.20,0.01,0.02,0.01
std,0.03,0.01,0.27,0.03,0.05,0.04
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.05,0.00,0.00,0.00
75%,0.01,0.00,0.29,0.00,0.01,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00


In [58]:
net_features_standard.describe().round(2)

,0,1,2,3,4,5
count,16482.00,16482.00,16482.00,16482.00,16482.00,16482.00
mean,-0.00,-0.00,-0.00,0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.37,-0.07,-0.72,-0.26,-0.34,-0.21
25%,-0.37,-0.07,-0.72,-0.26,-0.34,-0.21
50%,-0.37,-0.07,-0.54,-0.23,-0.34,-0.20
75%,-0.03,-0.07,0.33,-0.10,-0.19,-0.12
max,29.08,86.94,2.94,31.49,21.73,27.74


In [59]:
net_features_normalize.describe().round(2)

,0,1,2,3,4,5
count,16482.00,16482.00,16482.00,16482.0,16482.0,16482.0
mean,0.19,0.01,0.90,0.0,0.0,0.0
std,0.32,0.06,0.22,0.0,0.0,0.0
min,0.00,0.00,0.00,0.0,0.0,0.0
25%,0.00,0.00,0.94,0.0,0.0,0.0
50%,0.00,0.00,1.00,0.0,0.0,0.0
75%,0.32,0.00,1.00,0.0,0.0,0.0
max,1.00,0.99,1.00,0.0,0.0,0.0


In [60]:
net_features_log.describe().round(2)

,0,1,2,3,4,5
count,16482.00,16482.00,16482.00,16482.0,16482.0,16482.00
mean,0.63,0.03,1.46,0.0,0.0,0.00
std,1.00,0.24,0.79,0.0,0.0,0.00
min,0.00,0.00,0.69,0.0,0.0,0.00
25%,0.00,0.00,0.69,0.0,0.0,0.00
50%,0.00,0.00,1.10,0.0,0.0,0.00
75%,1.28,0.00,2.08,0.0,0.0,0.00
max,5.41,5.11,3.14,0.0,0.0,0.01
